# **準備**

ドライブのマウント

In [ ]:
#ドライブのマウント
from google.colab import drive
drive.mount('/content/drive')

必要なモジュールのインポート・画像ファイルパスの指定

In [ ]:
#モジュールのインポート
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 共通のディレクトリパス
common_path = '/content/drive/MyDrive/img2025/image/'

# **空間周波数スペクトルと画像の関係**

In [ ]:
#空間周波数スペクトルの生成 例）x軸周波数が4のとき
spectrum = np.array([    [0,   0,   0,   0,   1,   0,   0,   0],
                         [0,   0,   0,   0,   0,   0,   0,   0],
                         [0,   0,   0,   0,   0,   0,   0,   0],
                         [0,   0,   0,   0,   0,   0,   0,   0],
                         [0,   0,   0,   0,   0,   0,   0,   0],
                         [0,   0,   0,   0,   0,   0,   0,   0],
                         [0,   0,   0,   0,   0,   0,   0,   0],
                         [0,   0,   0,   0,   0,   0,   0,   0],
                         ], dtype=np.complex128)

# 2次元逆フーリエ変換を適用して画像を生成
image = np.fft.ifft2((spectrum)).real

#振幅スペクトルの計算
amp_spectrum =np.abs(spectrum)

#振幅スペクトルの周波数シフト
shifted_amp_spectrum = np.fft.fftshift(amp_spectrum)

# 画像を表示
plt.figure(figsize=(9, 3))

# 周波数スペクトルの表示（四隅が低周波，(0,0)が直流成分）
plt.subplot(1, 3, 1)
plt.title('Frequency Spectrum')
plt.imshow(amp_spectrum, cmap='gray')

# 周波数スペクトルの表示（中心が低周波，(4,4)が直流成分）
plt.subplot(1, 3, 2)
plt.title('Frequency Spectrum(Shifted)')
plt.imshow(shifted_amp_spectrum, cmap='gray')

plt.subplot(1, 3, 3)
plt.title('Reconstructed Image')
plt.imshow(image, cmap='gray')

plt.tight_layout()
plt.show()


# **自然画像の振幅スペクトル**

In [ ]:
# 画像を読み込む
gray_img = cv2.imread(common_path + 'gray_image.png', cv2.IMREAD_GRAYSCALE)

# 2D-FFT


#振幅スペクトルの計算


# 実行結果の表示
plt.figure(figsize=(6, 3))

# 自然画像
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(gray_img, cmap='gray', vmin=np.min(gray_img),vmax=np.max(gray_img))
plt.axis('off')

# 振幅画像（4隅が低周波）
plt.subplot(1, 2, 2)
plt.title('Frequency Spectrum')
plt.imshow(amp_spectrum, cmap='gray',vmin=np.min(amp_spectrum),vmax=np.max(amp_spectrum))
plt.axis('off')

plt.tight_layout()
plt.show()

# **2次元逆FFTによる画像の再構成**

In [ ]:
# 画像を読み込む
gray_img = cv2.imread(common_path + 'gray_image.png', cv2.IMREAD_GRAYSCALE)

# 2次元高速フーリエ変換（2D-FFT）
fourier_spectrum = (np.fft.fft2(gray_img))

# 2次元逆FFT（2D-IFFT）を適用し，実部を取り出す
reconstructed_img = np.fft.ifft2(fourier_spectrum).real

# 実行結果の表示
plt.figure(figsize=(6, 3))

# 自然画像
plt.subplot(1, 2, 1)
plt.title('Original Image')
plt.imshow(gray_img, cmap='gray', vmin=np.min(gray_img),vmax=np.max(gray_img))
plt.axis('off')

# 再構成画像
plt.subplot(1, 2, 2)
plt.title('Reconstructed Image')
plt.imshow(reconstructed_img, cmap='gray',vmin=np.min(reconstructed_img),vmax=np.max(reconstructed_img))
plt.axis('off')

plt.tight_layout()
plt.show()

# **ローパスフィルタによる周波数フィルタリング**

ローパスフィルタを生成する関数

In [ ]:
def generate_low_pass_filter(shape, radius):

    # フィルタの高さと幅を取得
    rows, cols = shape

    # 画像の中心座標を計算
    center_row, center_col = rows // 2, cols // 2

    # 全てのピクセルに対して、中心からの距離を求めるために座標を作成
    i, j = np.ogrid[0:rows, 0:cols]

    # 中心からの距離の二乗を計算
    squared_distance = (i - center_row)**2 + (j - center_col)**2

    # マスクを0で初期化
    mask = np.zeros(shape)

    # 距離の二乗が半径の二乗以下の要素に1を設定
    mask[squared_distance <= radius**2] = 1

    return mask

周波数フィルタリング

In [ ]:
# 画像を読み込む
gray_img = cv2.imread(common_path + 'gray_image.png', cv2.IMREAD_GRAYSCALE)

# 2D-FFTと周波数シフト

# フィルタリングを行う画像の画素数取得
shape = gray_img.shape

# 透過領域の半径
radius = 32

#ローパスフィルタの生成


# ローパスフィルタの適用


# 周波数シフトをしてから逆FFTをかけて実部を取り出す


#フィルタリング後の振幅スペクトル
amp_filtered_spectrum = np.log10(np.abs(filtered_spectrum)+1)

# 実行結果の表示
plt.figure(figsize=(9, 3))

# 原画像
plt.subplot(1, 3, 1)
plt.title('Original Image')
plt.imshow(gray_img, cmap='gray', vmin=np.min(gray_img),vmax=np.max(gray_img))
plt.axis('off')

#フィルタリング後の振幅スペクトルの表示
plt.subplot(1, 3, 2)
plt.title('Filtered Spectrum')
plt.imshow(amp_filtered_spectrum, cmap='gray',vmin=np.min(amp_filtered_spectrum),vmax=np.max(amp_filtered_spectrum))
plt.axis('off')

# 周波数フィルタリング後の画像
plt.subplot(1, 3, 3)
plt.title('Filtered Image')
plt.imshow(filtered_img, cmap='gray',vmin=np.min(filtered_img),vmax=np.max(filtered_img))
plt.axis('off')

plt.tight_layout()
plt.show()